# Agent Engine - Test Your Remote Agent

This notebook demonstrates how to test your agent after deploying it to Agent Engine.
You can:
- Load your deployed agent using the agent engine ID from deployment
- Send test queries and see responses
- Stream responses for real-time testing
- Test the ability of the agent to register feedback

The agent is defined in the [app/agent_engine_app.py](../app/agent_engine_app.py) file.


## Getting Started


### Install Vertex AI SDK for Python

Install the latest version of the Vertex AI SDK for Python as well as extra dependencies related to Reasoning Engine and LangChain:

In [ ]:
%pip install --upgrade --quiet "google-cloud-aiplatform[reasoningengine]" 

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Import libraries

In [1]:
import json
from vertexai.preview import reasoning_engines

### Connect to the remote Agent Engine

In [2]:
# Either specify agent engine ID manually or load from config file
REMOTE_AGENT_ENGINE_ID = ""  # Manually specify ID here if desired

# Load agent engine ID from deployment metadata if not specified manually
if REMOTE_AGENT_ENGINE_ID == "":
    with open("../deployment_metadata.json") as f:
        config = json.load(f)
        REMOTE_AGENT_ENGINE_ID = config["remote_agent_engine_id"]

reasoning_engine = reasoning_engines.ReasoningEngine(REMOTE_AGENT_ENGINE_ID)

### Sample input definition

We can define a sample input we will use to query our agent.

In [3]:
inputs = {
    "messages": [
        {"role": "user", "content": "Hello, AI!"},
        {"type": "ai", "content": "Hello!"},
        {"type": "user", "content": "What is the exchange rate from US dollars to Swedish currency?"},
    ]
}

### Test the agent
We are ready to test the remote agent by sending a synchronous query. The query will block until the response is received.
The response will contain the agent's reply to the input messages.

In [ ]:
reasoning_engine.query(input=inputs)

You can also stream responses from the agent.
This is useful for long-running queries where you want to see partial results
The stream_query method yields chunks of the response as they become available

In [ ]:
for x in reasoning_engine.stream_query(input=inputs): 
    print(x)

### Test the feedback collection method

Finally, we can test the `register_feedback` method we define in the Agent Application. 

In [6]:
feedback = {
    "score": 1.0,
    "text": "This was helpful",
    "run_id": "test-run-123",
    "log_type": "feedback"
}
reasoning_engine.register_feedback(feedback=feedback)